## Save as checkpoint

In [ ]:
import tensorflow as tf

In [ ]:
class Net(tf.keras.Model):
  def __init__(self):
    super(Net, self).__init__()
    self.l1 = tf.keras.layers.Dense(5)

  def call(self, x):
    return self.l1(x)

In [ ]:
net = Net()

In [ ]:
net.save_weights('easy_checkpoint')

In [ ]:
%%bash
ls

In [ ]:
def toy_dataset():
  inputs = tf.range(10.)[:, None] # None means shape=(10, 1)
  labels = inputs * 5. + tf.range(5.)[None, :]
  return tf.data.Dataset.from_tensor_slices(
    dict(x=inputs, y=labels)).repeat().batch(2)

In [ ]:
def train_step(net, example, optimizer):
  with tf.GradientTape() as tape:
    output = net(example['x'])
    loss = tf.reduce_mean(tf.abs(output - example['y']))
  variables = net.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return loss

In [ ]:
opt = tf.keras.optimizers.Adam(0.1)
dataset = toy_dataset()
iterator = iter(dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

In [ ]:
# Save checkpoint at every step
def train_and_checkpoint(net, manager):
  ckpt.restore(manager.latest_checkpoint)
  if manager.latest_checkpoint:
    print('restored from {}'.format(manager.latest_checkpoint))
  else:
    print('from scratch')

  for _ in range(50):
    example = next(iterator)
    loss = train_step(net, example, opt)
    ckpt.step.assign_add(1) # number of steps
    if int(ckpt.step) % 10 == 0:
      save_path = manager.save()
      print('Saved ckpt for step {}: {}'.format(int(ckpt.step), save_path))
      print('loss {:1.2f}'.format(loss.numpy()))

In [ ]:
train_and_checkpoint(net, manager)

## Restore and training

In [ ]:
opt = tf.keras.optimizers.Adam(0.1)
net = Net()
dataset = toy_dataset()
iterator = iter(dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

train_and_checkpoint(net, manager)

In [ ]:
print(manager.checkpoints)